In [1]:
import numpy as np
import gensim
import re
import os
import nltk
from sklearn.cluster import KMeans
from functools import reduce
from build_sentence_corp import extract_sent
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
import random
import pickle
import string

## Word2Vec

In [2]:
embedding_vector_size = 300
embedding_path = 'models/word2vec/GoogleNews-vectors-negative300.bin'
embedding = gensim.models.KeyedVectors.load_word2vec_format(embedding_path, binary=True)

In [3]:
def generate_vocab(sentence_dir):
    table = str.maketrans('','', string.punctuation)
    vocab_count = {}
    for sent in sentence_dir:
        for word in sent.split():
            word = word.lower().translate(table).strip()
            if word in vocab_count:
                vocab_count[word] += 1
            else:
                vocab_count[word] = 1
        
    vocab = set()
    k = 10    
    for key, val in vocab_count.items():
        if val >= k and key not in stopwords.words('english'):
            vocab.add(key)
    
    return vocab

In [4]:
def extract_sent(path, regex):
    review_dir = []
    for file in os.listdir(path):
        with open(path+file) as f:
            txt = f.read()
            reviews = re.findall(regex, txt)
        review_dir += reviews
    
    sentence_dir = []
    for review in review_dir:
        sentences = nltk.sent_tokenize(review)
        sentence_dir += sentences
        
    return sentence_dir

In [ ]:
def initializeT(words):
    matrix = np.empty((0, embedding_vector_size))
    for w in words:
        try:
            matrix = np.vstack((matrix, embedding[w]))
        except:
            pass
    print(matrix.shape)
    kmeans = KMeans(n_clusters=14, random_state=0).fit(matrix)
    return kmeans


In [ ]:
text_path = './data/TripAdvisor/Texts/'
regex = r"<Content>(.*)\n<Date>" 
sentence_dir = extract_sent(text_path, regex)

In [ ]:
len(sentence_dir)

2167783

In [ ]:
words = generate_vocab(sentence_dir)

In [ ]:
kmeans = initializeT(words)

In [76]:
pickle_T = open('T_matrix.pickle', 'wb')
pickle.dump(kmeans.cluster_centers_, pickle_T)
pickle_T.close()

In [78]:
pickle_in = open('T_matrix.pickle', 'rb')
centers = pickle.load(pickle_in)